# Resolución de CSPs con Propagación de Restricciones mediante Consistencia de Arcos

Aparte de la búsqueda en el espacio de estados, la programación con restricciones nos permite realizar un tipo específico de inferencia llamado **Propagación de Restricciones**, que ayuda a eliminar partes no deseadas del espacio de estados basándose en la satisfacción de las restricciones.

Una red es **consistente en nodos** (node-consistent) si todas las variables en ella también son consistentes en nodos, es decir, se satisfacen las restricciones unarias sobre todas las variables. También podríamos extender esta misma definición a las restricciones binarias, en cuyo caso la red se llamaría **consistente en arcos** (arc-consistent).

Podemos descomponer cualquier CSP, como el problema de colorear un mapa, en las siguientes secciones clave:

### 1. Variables, dominios y vecinos
Son los elementos a los que buscamos asignar un valor o estado aceptable.
Las variables son las **regiones** individuales del mapa que deben ser coloreados. En este caso se identificarán por las siglas de sus estados en una lista correspondientes a la siguiente imagen:


 <center>
        <img src="imgs/map_no_color.png" alt="Mapa de ejemplo para Coloreo" >
    </center>


In [1]:
from collections import deque
from util import *
from test_utils import *
from public_tests import *

In [2]:
variables = ["WA", "NT", "SA", "Q", "NSW", "V", "T"]

Los dominios para programar la solución por consistencia de arcos, comenzarán con los tres colores posibles para cada región.

In [3]:
dominios = {var: ["rojo", "verde", "azul"] for var in variables}
print(dominios)

{'WA': ['rojo', 'verde', 'azul'], 'NT': ['rojo', 'verde', 'azul'], 'SA': ['rojo', 'verde', 'azul'], 'Q': ['rojo', 'verde', 'azul'], 'NSW': ['rojo', 'verde', 'azul'], 'V': ['rojo', 'verde', 'azul'], 'T': ['rojo', 'verde', 'azul']}


Para definir las restricciones definiremos un grafo llamado vecinos como una estructura de datos de tipo diccionario que permita representar a los estados y a sus vecinos.

In [4]:
vecinos = {
    "WA": ["NT", "SA"],
    "NT": ["WA", "SA", "Q"],
    "SA": ["WA", "NT", "Q", "NSW", "V"],
    "Q":  ["NT", "SA", "NSW"],
    "NSW":["Q", "SA", "V"],
    "V":  ["SA", "NSW"],
    "T":  []
}

### 2.Implementación de `remove_inconsistent_values(Xi, Xj, dominios)`

Esta función implementa el **procedimiento `REMOVE-INCONSISTENT-VALUES(Xi, Xj)`** del algoritmo **AC-3**.  Su objetivo es **eliminar valores del dominio de una variable `Xi`** que sean inconsistentes con la variable `Xj`.

<center>
        <img src="imgs/ac3.png" alt="AC-3" >
    </center>

En los **problemas de satisfacción de restricciones (CSP)**, la consistencia de arcos significa que **para cada valor de `Xi` debe existir al menos un valor posible en `Xj`** que no viole las restricciones binarias (por ejemplo, que `Xi` y `Xj` no tomen el mismo color en el problema de coloreo de mapas). Si no existe tal valor, el valor de `Xi` se elimina de su dominio.

#### Lógica paso a paso

1. Recorrer cada valor `x` en el dominio de `Xi`.
2. Verificar si existe **algún valor `y` en el dominio de `Xj`** que sea **compatible** con `x`.
3. Si no existe tal `y`, eliminar `x` del dominio de `Xi` porque **no puede formar parte de una solución consistente**.
4. Retornar `True` si se eliminó al menos un valor; de lo contrario, retornar `False`.

Completa la implementación de remove_inconsistent_values:


In [5]:
def remove_inconsistent_values(Xi, Xj, dominios):
    """
    Implementa la función REMOVE-INCONSISTENT-VALUES(Xi, Xj)
    del pseudocódigo de AC-3.

    Parámetros:
        Xi, Xj : variables involucradas en la restricción binaria.
        dominios : diccionario { variable: [valores posibles] }.

    Retorna:
        True si se eliminaron valores del dominio de Xi,
        False
    """
    removed = False
    
    for x in dominios[Xi]: #completar recorrer cada valor x en el dominio de Xi.        
        # entonces (Xi, Xj) es inconsistente bajo esa asignación.
        if not any(x != y for y in dominios[Xj]): # Si ningún valor y en el dominio de Xj es diferente de x,
            dominios[Xi].remove(x) #elimina x del dominio de Xi
            removed = True #si se eliminó al menos un valor retorna True
    return removed

In [6]:
def run_remove_inconsistent_tests(cases):
    tests = []

    for case in cases:
        Xi = case["input"]["Xi"]
        Xj = case["input"]["Xj"]
        dominios = {k: v[:] for k, v in case["input"]["dominios"].items()}  # copia segura

        result = remove_inconsistent_values(Xi, Xj, dominios)
        
        # Si hay propagaciones indicadas, aplicarlas secuencialmente
        for arc in case["input"].get("propagation", []):
            remove_inconsistent_values(arc[0], arc[1], dominios)
        
        ok = (result == case["expected_result"] and dominios == case["expected_dominios"])
        tests.append((ok, case["desc"]))

    # ------------------------------------------------
    #     Resultados
    # ------------------------------------------------
    passed = sum(1 for ok, _ in tests if ok)
    total = len(tests)
    
    print("=======================================")
    print("🔍 RESULTADOS DE LAS PRUEBAS")
    print("=======================================")
    for ok, desc in tests:
        status = "✅ OK" if ok else "❌ FALLÓ"
        print(f"{status} - {desc}")
    print("=======================================")
    print(f"✔️ {passed}/{total} pruebas superadas.")
    print("=======================================")

# Ejecutar pruebas
run_remove_inconsistent_tests(test_cases_remove_inconsistent_values)



🔍 RESULTADOS DE LAS PRUEBAS
✅ OK - Caso 1: Eliminación simple
✅ OK - Caso 2: Sin conflicto
✅ OK - Caso 3: Dominio vacío tras eliminación
✅ OK - Caso 4: Al menos un valor compatible
✅ OK - Caso 5: Propagación de restricción a Xk
✔️ 5/5 pruebas superadas.


## 3. Programando el Algoritmo AC-3

El **algoritmo AC-3 (Arc Consistency 3)** es una técnica fundamental en los **Problemas de Satisfacción de Restricciones (CSP)**, como el **coloreo de mapas**.  Su objetivo es **reducir los dominios de las variables** eliminando valores inconsistentes antes (o durante) la búsqueda, ayudando a **podar el espacio de búsqueda**. El algoritmo de AC-3 puede implementarse de la siguiente manera:

1. **Inicializar una cola** con todos los arcos del problema:  
   pares ordenados `(Xi, Xj)` donde `Xi` y `Xj` son vecinos (tienen una restricción entre sí).

2. Mientras la cola **no esté vacía**:
   - Sacar un arco `(Xi, Xj)` de la cola.
   - Llamar a `remove_inconsistent_values(Xi, Xj, dominios)`:
     - Si elimina algún valor del dominio de `Xi`, entonces debemos revisar todos los arcos `(Xk, Xi)` donde `Xk` también es vecino de `Xi` (excepto `Xj`).
     - Si un dominio queda vacío, **no hay solución posible**.

3. Si terminamos sin vaciar ningún dominio completamente, el problema está **arc-consistente**.

Completa la implementación de ac3:


In [7]:
def ac3(variables, dominios, vecinos):
    """
    Implementa el algoritmo AC-3 para lograr consistencia de arcos.

    Parámetros:
        variables : lista de variables del CSP (por ejemplo, regiones del mapa).
        dominios  : diccionario { variable: [valores posibles] }.
        vecinos   : diccionario { variable: [otras variables con restricciones binarias] }.

    Retorna:
        True si los dominios son consistentes.
        False si algún dominio queda vacío (no hay solución posible).
    """
    # Inicializamos la cola con todos los arcos (Xi, Xj)
    queue = deque((Xi, Xj) for Xi in variables for Xj in vecinos[Xi])

    while queue:
        Xi, Xj = queue.popleft() #saca un arco de la cola

        if remove_inconsistent_values(Xi, Xj, dominios): #remueve valores inconsistentes dados Xi, Xj, dominios
            if not dominios[Xi]: #Si no quedan valores posibles en Xi
                return False  # No hay solución posible

            for Xk in vecinos[Xi]: #evalua vecinos de Xi 
                if Xk != Xj: #si Xk diferente a Xj
                    queue.append((Xk, Xi)) #propaga la restriccion agregando a la cola la tupla (Xk, Xi)
    
    return True

In [8]:
def run_remove_inconsistent_tests(cases):
    """
    Ejecuta todos los casos de prueba definidos en test_cases_remove_inconsistent_values.
    """
    tests = []

    for case in cases:
        Xi = case["input"]["Xi"]
        Xj = case["input"]["Xj"]
        dominios = {k: v[:] for k, v in case["input"]["dominios"].items()}  # copia profunda

        result = remove_inconsistent_values(Xi, Xj, dominios)

        # Caso especial: propagación de restricciones (A -> B -> C)
        if case["input"].get("propagacion"):
            _ = remove_inconsistent_values("C", "B", dominios)

        ok = (result == case["expected_result"] and dominios == case["expected_dominios"])
        tests.append((ok, case["desc"]))

    # ------------------------------------------------
    #  Resultados
    # ------------------------------------------------
    passed = sum(1 for ok, _ in tests if ok)
    total = len(tests)
    
    print("=======================================")
    print("🔍 RESULTADOS DE LAS PRUEBAS")
    print("=======================================")
    for ok, desc in tests:
        status = "✅ OK" if ok else "❌ FALLÓ"
        print(f"{status} - {desc}")
    print("=======================================")
    print(f"✔️ {passed}/{total} pruebas superadas.")
    print("=======================================")

# Ejecutar las pruebas
run_remove_inconsistent_tests(test_cases_remove_inconsistent_values_ac_3)


🔍 RESULTADOS DE LAS PRUEBAS
✅ OK - Caso 1: No debe eliminar valores si no hay conflicto.
✅ OK - Caso 2: Debe eliminar el valor 'rojo' de A.
✅ OK - Caso 3: Xi debe quedar vacío si todos los valores son inconsistentes.
✅ OK - Caso 4: No debe eliminar valores si hay al menos un color distinto en B.
✅ OK - Caso 5: Debe eliminar 'azul' de A, dejando solo 'rojo'.
✅ OK - Caso 6: Propagación correcta A -> B -> C (B pierde 'rojo', luego C pierde 'verde').
✔️ 6/6 pruebas superadas.


### ¿Qué hace AC-3 en la práctica?

- **Elimina valores inconsistentes** antes de usar backtracking.  
- **Reduce el espacio de búsqueda**, haciendo que la búsqueda sea más eficiente.  
- **No garantiza una solución completa**, pero deja los dominios lo más “limpios” posible.  



## 4. Conexión con Backtracking

El **AC-3** puede integrarse con **Backtracking** de dos formas:

**Antes de iniciar la búsqueda:**  
   Se ejecuta para reducir los dominios iniciales y evitar ramas inútiles. Puedes ejecutar la celda a continuación para observar como se filtran los dominios y luego mediante backtracking se encuentra la solución. Al inicializar "WA" en azul, se elimina "azul" de NT y SA. Observa los print.


In [9]:
# Aplicar AC-3
dominios = {var: ["rojo", "verde", "azul"] for var in variables}
dominios["WA"] = ["azul"]
print("dominios antes:", dominios)
ac3(variables, dominios, vecinos)
print("dominios después de AC-3:", dominios)
# Aplicar Backtracking
solucion = backtracking({}, dominios, vecinos, variables)

if solucion:
    print("\n✅ Solución encontrada con AC-3 + Backtracking:\n")
    for var, color in solucion.items():
       print(f"{var}: {color}")
else:
    print("❌ No se encontró solución con el dominio de colores dado.")

dominios antes: {'WA': ['azul'], 'NT': ['rojo', 'verde', 'azul'], 'SA': ['rojo', 'verde', 'azul'], 'Q': ['rojo', 'verde', 'azul'], 'NSW': ['rojo', 'verde', 'azul'], 'V': ['rojo', 'verde', 'azul'], 'T': ['rojo', 'verde', 'azul']}
dominios después de AC-3: {'WA': ['azul'], 'NT': ['rojo', 'verde'], 'SA': ['rojo', 'verde'], 'Q': ['rojo', 'verde', 'azul'], 'NSW': ['rojo', 'verde', 'azul'], 'V': ['rojo', 'verde', 'azul'], 'T': ['rojo', 'verde', 'azul']}

✅ Solución encontrada con AC-3 + Backtracking:

WA: azul
NT: rojo
SA: verde
Q: azul
NSW: rojo
V: azul
T: rojo


**Durante la búsqueda:**  
   Se aplica después de cada asignación parcial para **mantener la consistencia local**.

Estos algoritmos combinan **AC-3 + Backtracking**, para definir `bracktrack_mac` (backtracking maintaining arc consistency) logrando búsquedas más **eficientes** en problemas grandes de **Satisfacción de Restricciones (CSP)**. Puedes ejecutar la celda a continuación para ver como se combinan AC3 y bracktracking.


In [10]:
def backtrack_mac(asignacion, vecinos, dominios):
    """Implementa Backtracking con AC-3 tras cada asignación."""
    # Si todas las regiones están asignadas, se ha encontrado solución    
    if len(asignacion) == len(vecinos):
        return asignacion

    # Seleccionar siguiente región sin asignar
    region = [v for v in vecinos if v not in asignacion][0]

    for color in dominios[region]:
        if es_consistente(region, color, asignacion, vecinos):
            # Asignamos temporalmente
            asignacion[region] = color
            # Clonamos dominios y reducimos según la asignación actual
            nuevos_dominios = {v: list(dominios[v]) for v in dominios}
            nuevos_dominios[region] = [color]
            # Aplicamos AC-3 localmente
            if ac3(asignacion, nuevos_dominios, vecinos):
                resultado = backtrack_mac(asignacion, vecinos, nuevos_dominios) 
               
                if resultado:
                    return resultado
                    
            # Retroceso
            del asignacion[region]
    return None

In [11]:
# --- Ejecución del algoritmo ---
dominios = {var: ["rojo", "verde", "azul"] for var in variables}
dominios["WA"] = ["verde"]
solucion = backtrack_mac({}, vecinos, dominios)

# --- Mostrar resultado ---
if solucion:
    print("✅ Solución encontrada con Backtracking + AC-3 (MAC):")
    for region, color in solucion.items():
        print(f"   {region}: {color}")
else:
    print("❌ No se encontró una solución.")

✅ Solución encontrada con Backtracking + AC-3 (MAC):
   WA: verde
   NT: rojo
   SA: azul
   Q: verde
   NSW: rojo
   V: verde
   T: rojo


## Conclusiones

1. **AC-3 elimina valores inconsistentes** de los dominios antes o durante la búsqueda, reduciendo el espacio de soluciones posibles.  
2. La función `remove_inconsistent_values()` es clave para mantener la **consistencia de arcos**, eliminando valores que no tienen soporte en las variables vecinas.   
4. Combinado con Backtracking, AC-3 mejora la eficiencia sin sacrificar la corrección del proceso de búsqueda.


## Referencias

[1] Russell, S., & Norvig, P.: **Artificial Intelligence: A Modern Approach**. Pearson, Upper Saddle River, 4th edition, 2020, ISBN 978-0134610993.

[2] **Material didáctico y ejercicios sobre Problemas de Satisfacción de Restricciones (CSP)**.
[https://inst.eecs.berkeley.edu/~cs188/textbook/csp/](https://inst.eecs.berkeley.edu/~cs188/textbook/csp/)

[3] **Ejemplo de código para el problema de coloreado de mapas (CSP)**.
[https://github.com/ahforoughi/map_coloring_csp/tree/main](https://github.com/ahforoughi/map_coloring_csp/tree/main)